In [1]:
import openreview
import re
import unicodecsv as csv
import os
import requests

In [2]:
client = openreview.Client(username="OpenReview.net",password="OpenReview_beta",baseurl="http://openreview.net")

outputdir = './tpms-dump-final'

In [3]:
def get_profiles(groupname):
    print "groupname:",groupname
    members = client.get_group(groupname).members
    
    p = re.compile('~.*')

    canonical_ids = []
    missing_users = []
    for m in members:
        m = m.strip()
        try:
            print('m:',m)
            client.get_group(m)
            g = client.get_group(m)
            groupmembers = [member for member in g.members if p.match(member)]

            if len(groupmembers) ==0:
                print "No canonical IDs found for member ",m
                missing_users.append(m)
            else:
                if len(groupmembers) > 1:
                    print "More than one canonical ID found for member ",m,"; Using first ID in the list."

                tildeId = groupmembers[0]
                canonical_ids.append(tildeId)

        except openreview.OpenReviewException as e:
            missing_users.append(m)
            print e

    profiles = []

    for id in canonical_ids:
        print "id: ",id
        profiles.append(client.get_note(id).to_json())
    
    return profiles,missing_users

In [4]:
def dump_names(profiles,outfilename,missingTuple=None):
    if not os.path.exists(outputdir):
        os.makedirs(outputdir)

    with open(outputdir+'/'+outfilename,'wb') as outfile:
        csvwriter = csv.writer(outfile, delimiter=',')
        for profile in profiles:
            p = profile['content']
            name = p['names'][0]
            firstname = name['first']
            lastname = name['last']
            email = p['emails'][0]
            csvwriter.writerow([email,firstname,lastname])
        if missingTuple !=None:
            missingList = missingTuple[0]
            missingMap = missingTuple[1]
            for email in missingList:
                try:
                    csvwriter.writerow([email,missingMap[email][0],missingMap[email][1]])
                except KeyError as e:
                    print "Key error found on email ",email,': ',e

In [5]:
def write_pdfs(papers):
    external = re.compile('http.*')
    internal = re.compile('\/pdf\/.*')
    for s in papers:
        if external.match(s['content']['pdf']):
            r = requests.get(s['content']['pdf'])
        elif(internal.match(s['content']['pdf'])):
            r = requests.get(client.baseurl+'/pdf?id='+s['id'], headers=client.headers)
        else:
            print "Couldn't get PDF for note ",s['id']
            
        print "number",s['number'],"response",r
        if not os.path.exists(outputdir+'/pdfs'):
            os.makedirs(outputdir+'/pdfs')
        with open(outputdir+'/pdfs/paper'+str(s['number'])+'.pdf', 'wb') as f:
            f.write(r.content)
            f.close()

In [6]:
def get_conflicts(profile):
    conflicts = set()
    for h in profile['content']['history']:
        conflicts.add(h['institution']['domain'])
    for e in profile['content']['emails']:
        domain = e.split('@')[1]
        conflicts.add(domain)
    return conflicts

In [7]:
def dump_conflicts(profiles, missing, papers, outfilename):
    with open(outputdir+'/'+outfilename,'wb') as outfile:
        csvwriter=csv.writer(outfile,delimiter=',')
        for profile in profiles:
            email = profile['content']['emails'][0]
            profile_conflicts = get_conflicts(profile)
            for paper in papers:
                paper_id = str(paper['number'])
                if not profile_conflicts.isdisjoint(paper['content']['conflicts']):
                    print "conflict detected:",email,profile_conflicts,paper['content']['conflicts']
                    csvwriter.writerow([paper_id,email])
        
        for email in missing:
            domain = email.split('@')[1]
            for paper in papers:
                paper_id = str(paper['number'])
                if domain in paper['content']['conflicts']:
                    csvwriter.writerow([paper_id,email])

In [8]:
def dump_missing(list):
    with open(outputdir+'/missing-dump.csv','wb') as outfile:
        for email in list:
            csvwriter=csv.writer(outfile,delimiter=',')
            first = all_reviewers[email][0]
            last = all_reviewers[email][1]
            csvwriter.writerow([])

In [9]:
client.user

{'id': 'OpenReview.net', 'password': 'OpenReview_beta'}

In [10]:
reviewer_profiles,missing_reviewers = get_profiles("ICLR.cc/2017/conference/reviewers")

groupname: ICLR.cc/2017/conference/reviewers
('m:', u'williamchan@cmu.edu')
('m:', u'jimmy@psi.toronto.edu')
('m:', u'kmh@google.com')
('m:', u'edouard.grave@gmail.com')
('m:', u'guillaume.desjardins@gmail.com')
('m:', u'jaakko.peltonen@aalto.fi')
[{u'path': u'id', u'type': u'Not Found', u'value': u'jaakko.peltonen@aalto.fi'}]
('m:', u'christoph.lassner@tuebingen.mpg.de')
('m:', u'benenson@mpi-inf.mpg.de')
[{u'path': u'id', u'type': u'Not Found', u'value': u'benenson@mpi-inf.mpg.de'}]
('m:', u'tt@cs.dal.ca')
('m:', u'deigen@cs.nyu.edu')
('m:', u'heng.g.luo@gmail.com')
('m:', u'dkale@usc.edu')
('m:', u'edouard.oyallon@ens.fr')
('m:', u'urtasun@cs.toronto.edu')
('m:', u'mahadeva@cs.umass.edu')
[{u'path': u'id', u'type': u'Not Found', u'value': u'mahadeva@cs.umass.edu'}]
('m:', u'eric.jy.xie@gmail.com')
('m:', u'faffne@gmail.com')
('m:', u'djw500@gmail.com')
[{u'path': u'id', u'type': u'Not Found', u'value': u'djw500@gmail.com'}]
('m:', u'peter.bell@ed.ac.uk')
('m:', u'wittawatj@gmail.com

In [11]:
reviewer_profiles

[{'active': True,
  'cdate': None,
  'content': {u'dblp': u'',
   u'emails': [u'williamchan@cmu.edu'],
   u'expertise': [{u'end': None,
     u'keywords': [u'deep learning', u'speech'],
     u'start': None}],
   u'gscholar': u'https://scholar.google.com/citations?user=Nla9qfUAAAAJ',
   u'history': [{u'end': None,
     u'institution': {u'domain': u'cmu.edu',
      u'name': u'Carnegie Mellon University'},
     u'position': u'PhD student',
     u'start': None}],
   u'homepage': u'',
   u'linkedin': u'',
   u'names': [{u'first': u'William',
     u'last': u'Chan',
     u'middle': u'',
     u'preferred': True,
     u'username': u'~William_Chan1'}],
   u'password': {u'hash': u'62AxypbxCOq/DqSwYpVZMWx3b85M1HHBrvCcFNOt531BhSFhmCelP+fVg3piDvNi32hJhFCahVmzEZ9cLTjFat5f',
    u'hashMethod': u'pbkdf2',
    u'keyLength': 66,
    u'salt': u'QlmbbDN3UpTbqnYnqj2F7NxOHoU4xJqlcJvtOSaQE1SAognYB6dHIy7qGKz7UPg0my',
    u'workUnits': 60},
   u'preferred_email': u'williamchan@cmu.edu',
   u'wikipedia': u''},
  

In [12]:
areachair_profiles,missing_areachairs = get_profiles("ICLR.cc/2017/areachairs")


groupname: ICLR.cc/2017/areachairs
('m:', u'srush@seas.harvard.edu')
('m:', u'a.anandkumar@uci.edu')
[{u'path': u'id', u'type': u'Not Found', u'value': u'a.anandkumar@uci.edu'}]
('m:', u'christopher.pal@polymtl.ca')
('m:', u'duvenaud@cs.toronto.edu')
('m:', u'parikh@vt.edu')
('m:', u'dbatra@vt.edu')
('m:', u'etg@google.com')
('m:', u'gdahl@google.com')
('m:', u'gwtaylor@uoguelph.ca')
('m:', u'i.murray@ed.ac.uk')
('m:', u'titov@mmci.uni-saarland.de')
('m:', u'jasper@cs.toronto.edu')
('m:', u'bruna@cims.nyu.edu')
[{u'path': u'id', u'type': u'Not Found', u'value': u'bruna@cims.nyu.edu'}]
('m:', u'simonyan@google.com')
('m:', u'lvdmaaten@gmail.com')
('m:', u'schmidtm@cs.ubc.ca')
('m:', u'aapo.hyvarinen@helsinki.fi')
[{u'path': u'id', u'type': u'Not Found', u'value': u'aapo.hyvarinen@helsinki.fi'}]
('m:', u'matthias@bethgelab.org')
('m:', u'ndjaitly@google.com')
('m:', u'nasmith@cs.washington.edu')
[{u'path': u'id', u'type': u'Not Found', u'value': u'nasmith@cs.washington.edu'}]
('m:', u'ra

In [13]:
missing_areachairs

[u'a.anandkumar@uci.edu',
 u'bruna@cims.nyu.edu',
 u'aapo.hyvarinen@helsinki.fi',
 u'nasmith@cs.washington.edu',
 u'sanja.fidler@gmail.com',
 u'slav@google.com',
 u'notebleue@gmail.com',
 u'zaid.harchaoui@inria.fr']

In [14]:
missingReviewerMap = {}
with open('./iclr_all_reviewers.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    c=0
    for row in reader:
        if c>0:
            print row
            missingReviewerMap[str(row[2])]=row[0:2]
        c+=1
        
missingAreachairMap = {}
with open('./iclr_area_chairs.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    c=0
    for row in reader:
        if c>0:
            print row
            missingAreachairMap[str(row[2])]=row[0:2]
        c+=1
      

[u'Guillaume', u'Alain', u'guillaume.alain.umontreal@gmail.com']
[u'Alexandre', u'Allauzen', u'allauzen@limsi.fr']
[u'Amjad', u'Almahairi', u'amjadmahayri@gmail.com']
[u'Ouais', u'Alsharif', u'ouaisa@google.com']
[u'Yali', u'Amit', u'amit@galton.uchicago.edu']
[u'Dragomir', u'Anguelov', u'dragomir@google.com']
[u'Itamar', u'Arel', u'itamar@ieee.org']
[u'Ludovic', u'Arnold', u'perso@ludovicarnold.com']
[u'Yusuf', u'Aytar', u'yusuf@robots.ox.ac.uk']
[u'Jimmy Lei', u'Ba', u'jimmy@psi.toronto.edu']
[u'Francis', u'Bach', u'francis.bach@inria.fr']
[u'Drew', u'Bagnell', u'bagnell2@andrew.cmu.edu']
[u'Pierre', u'Baldi', u'pfbaldi@ics.uci.edu']
[u'Nicolas', u'Ballas', u'ballas.n@gmail.com']
[u'Paul', u'Baltescu', u'pauldb89@gmail.com']
[u'Marco', u'Baroni', u'marco.baroni@unitn.it']
[u'Justin', u'Bayer', u'bayer.justin@googlemail.com']
[u'Peter', u'Bell', u'peter.bell@ed.ac.uk']
[u'James', u'Bergstra', u'james.bergstra@gmail.com']
[u'Misha', u'Bilenko', u'mbilenko@microsoft.com']
[u'Phil', u'Bl

In [15]:
papers = [s.to_json() for s in client.get_notes(invitation='ICLR.cc/2017/conference/-/submission')]

        
dump_names(reviewer_profiles,"reviewer-dump.csv",(missing_reviewers,missingReviewerMap))
dump_names(areachair_profiles,"areachair-dump.csv",(missing_areachairs,missingAreachairMap))
write_pdfs(papers)
dump_conflicts(reviewer_profiles,missing_reviewers+missing_areachairs,papers,"conflicts-dump.csv")

Key error found on email  drk@calicolabs.com :  u'drk@calicolabs.com'
Key error found on email  vijay.p@jhu.edu :  u'vijay.p@jhu.edu'
Key error found on email  pghahre1@jhu.edu :  u'pghahre1@jhu.edu'
Key error found on email  jason@cs.jhu.edu :  u'jason@cs.jhu.edu'
Key error found on email  grangier@fb.com :  u'grangier@fb.com'
Key error found on email  jamesk@cse.ust.hk :  u'jamesk@cse.ust.hk'
Key error found on email  sussillo@google.com :  u'sussillo@google.com'
Key error found on email  sandrafaust@google.com :  u'sandrafaust@google.com'
Key error found on email  springj@informatik.uni-freiburg.de :  u'springj@informatik.uni-freiburg.de'
number 610 response <Response [200]>
number 609 response <Response [200]>
number 608 response <Response [200]>
number 607 response <Response [200]>
number 606 response <Response [200]>
number 605 response <Response [200]>
number 604 response <Response [200]>
number 601 response <Response [200]>
number 600 response <Response [200]>
number 599 respon